In [2]:

import warnings
warnings.filterwarnings('ignore')
import datetime
from torch.utils.data import Dataset
from skimage.transform import resize

from zipfile import ZipFile
import os
import torch

import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from Dataset import FaceDataset,Scale,ToTensor
from Models import resblock,network_9layers,network_29layers,network_29layers_v2,save_model
from losses import TripletLoss
import skimage
from skimage.io import imread,imsave
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm 
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import os
import shutil
import random
import pandas as pd

In [17]:
a=torch.randn((32,256))
b=torch.matmul(a, a.T)
b2 = torch.diagonal(b,0)
print(a.size(),b.size(),b2.size())
distances =b2.unsqueeze(0)- 2.0 * b + b2.unsqueeze(1)
print(b2.unsqueeze(1).size(),distances.size())
print(distances)

torch.Size([32, 256]) torch.Size([32, 32]) torch.Size([32])
torch.Size([32, 1]) torch.Size([32, 32])
tensor([[  0.0000, 481.6660, 507.1423,  ..., 483.9155, 470.4411, 446.1161],
        [481.6660,   0.0000, 520.0730,  ..., 512.0319, 529.1748, 455.2528],
        [507.1423, 520.0730,   0.0000,  ..., 520.1583, 572.6542, 507.6129],
        ...,
        [483.9155, 512.0319, 520.1583,  ...,   0.0000, 517.0218, 437.5637],
        [470.4411, 529.1748, 572.6542,  ..., 517.0219,   0.0000, 486.9256],
        [446.1161, 455.2528, 507.6129,  ..., 437.5637, 486.9256,   0.0000]])


In [21]:
distances = torch.max(distances, torch.tensor([0.0]))
print(distances.size())
print(distances)

torch.Size([32, 32])
tensor([[12345556., 12345556., 12345556.,  ..., 12345556., 12345556.,
         12345556.],
        [12345556., 12345556., 12345556.,  ..., 12345556., 12345556.,
         12345556.],
        [12345556., 12345556., 12345556.,  ..., 12345556., 12345556.,
         12345556.],
        ...,
        [12345556., 12345556., 12345556.,  ..., 12345556., 12345556.,
         12345556.],
        [12345556., 12345556., 12345556.,  ..., 12345556., 12345556.,
         12345556.],
        [12345556., 12345556., 12345556.,  ..., 12345556., 12345556.,
         12345556.]])


# Generate Train and test data (Segregation)

In [1]:
df=pd.read_csv('image_data.csv')
print(len(df))
df_train=pd.read_csv('train.csv')
print(len(df_train))
df_test=pd.read_csv('test.csv')
print(len(df_test))

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


NameError: name 'pd' is not defined

In [ ]:

# label_list=list(set(df.Label))
# train_labels=random.sample(label_list,int(0.8*len(label_list)))
# test_labels=list(set(label_list)-set(train_labels))

# print(len(label_list),len(train_labels),len(test_labels))
# df_train=df[df.Label.isin(train_labels)]
# df_test=df[df.Label.isin(test_labels)]
# print(len(df),'\n',len(df_train),'\n',len(df_test))
# df_train.to_csv('train.csv',index=False)
# df_test.to_csv('test.csv',index=False)

# Light CNN


In [ ]:
transform=torchvision.transforms.Compose([Scale(),ToTensor()])

train_dataset=FaceDataset('img_align_celeba','train.csv',transform)
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_dataset=FaceDataset('img_align_celeba','test.csv',transform)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)
print(test_loader.__len__())
print(train_loader.__len__())

# PROBLEM WITH DATASET

In [ ]:
def visualize_loader(loader):
    for i,sample in enumerate(loader):
        #print(sample['image'].shape)
        if i==1:
            index=0
            
            image=(sample['image'][index]).numpy()
            
            
            #image=np.zeros(image_i.shape,dtype=np.uint8)
            #image[np.where(image_i!=0)]=255

            label=(sample['label'][index]).numpy()
            
            image=np.squeeze(image.transpose(1,2,0),axis=2)
            
            
            print(label)
            fig=plt.figure()
            plt.imshow(image,cmap='gray')
            
           
           
            break
visualize_loader(test_loader)

In [3]:
block=resblock
model=network_29layers_v2(block, [1, 2, 3, 4])

model_start_date=datetime.datetime.now().strftime("%Y_%m_%d")
BEST_MODEL_PATH=os.path.join(os.getcwd(),'model_{}'.format(model_start_date))
if not os.path.exists(BEST_MODEL_PATH):
    os.mkdir(BEST_MODEL_PATH)
    print('model_{} dir has been made'.format(model_start_date))
print("Model's state_dict:")
writer = SummaryWriter('model_{}/m2fpa_experiment_{}'.format(model_start_date,1))
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())



model_2020_01_16 dir has been made
Model's state_dict:
conv1.filter.weight 	 torch.Size([96, 1, 5, 5])
conv1.filter.bias 	 torch.Size([96])
block1.0.conv1.filter.weight 	 torch.Size([96, 48, 3, 3])
block1.0.conv1.filter.bias 	 torch.Size([96])
block1.0.conv2.filter.weight 	 torch.Size([96, 48, 3, 3])
block1.0.conv2.filter.bias 	 torch.Size([96])
group1.conv_a.filter.weight 	 torch.Size([96, 48, 1, 1])
group1.conv_a.filter.bias 	 torch.Size([96])
group1.conv.filter.weight 	 torch.Size([192, 48, 3, 3])
group1.conv.filter.bias 	 torch.Size([192])
block2.0.conv1.filter.weight 	 torch.Size([192, 96, 3, 3])
block2.0.conv1.filter.bias 	 torch.Size([192])
block2.0.conv2.filter.weight 	 torch.Size([192, 96, 3, 3])
block2.0.conv2.filter.bias 	 torch.Size([192])
block2.1.conv1.filter.weight 	 torch.Size([192, 96, 3, 3])
block2.1.conv1.filter.bias 	 torch.Size([192])
block2.1.conv2.filter.weight 	 torch.Size([192, 96, 3, 3])
block2.1.conv2.filter.bias 	 torch.Size([192])
group2.conv_a.filter.weigh

In [4]:
a=torch.ones((32,1,128,128))
b=model(a)
print(b.size())

torch.Size([32, 256])


In [ ]:
print(torch.cuda.get_device_name(torch.cuda.current_device()))
optimizer_selected='adam'
batchsize=4
no_steps=train_dataset.__len__()//batchsize
restart_epochs=8
num_epochs=10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


criterion=TripletLoss(margin=8.4,device=device ,squared=False,batch_all=False)

model = model.to(device)



if optimizer_selected=='adam':
    optimizer = torch.optim.Adam(model.parameters(),lr=10e-03, betas=(0.9, 0.98),weight_decay=0.02)
else:
    optimizer = torch.optim.SGD(model.parameters(),lr=10e-03, momentum=0.8,nesterov=True)

scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, restart_epochs*no_steps,\
                                                     eta_min=10e-012, last_epoch=-1)

best_val=10e06
for epoch in range(num_epochs):
    loop=tqdm(train_loader)
    print("Learning Rate : {}".format(optimizer.state_dict()['param_groups'][-1]['lr']))
    # loop over the dataset multiple times
    

    for mode in ['train','eval']:
     
        if mode == 'train':
            
            running_loss = 0.0
            model.train()

            for i, sample_batched in (enumerate(loop)):
                loop.set_description('Epoch {}/{}'.format(epoch + 1, num_epochs))
                #print("epoch {},iter {}".format(epoch,i))

                # zero the parameter gradients

                optimizer.zero_grad()
                # get the inputs; data is a list of [inputs, labels]
                image_batch, labels_batch = sample_batched['image'],sample_batched['label']
                
                image_batch, labels_batch = image_batch.to(device, dtype = torch.float)\
                ,labels_batch.to(device, dtype = torch.float)

                # forward + backward + optimize
                
                embeddings_train=model(image_batch)
                #outputs = torch.sigmoid()
                
                loss = criterion(labels_batch,embeddings_train,)
        
                loss.backward()
                optimizer.step()
                
                scheduler.step()

                    
                loop.set_postfix(train_loss=loss.detach().item())
                
                 
                    
        elif mode =='eval':
            optimizer.zero_grad()
            samples_test=len(test_loader)
            model.eval()
            final_test_loss=0
            
            for j, test_sample in enumerate(test_loader):
                
                optimizer.zero_grad()
                # get the inputs; data is a list of [inputs, labels]
                image_batch_test, label_batch_test = test_sample['image'],test_sample['label']
                image_batch_test, label_batch_test,neg_batch_test = image_batch_test.to(device, dtype = torch.float)\
                ,label_batch_test.to(device, dtype = torch.float)

                # forward + backward + optimize
                embeddings=model(image_batch_test)
                
                
                #outputs = torch.sigmoid()
                
                test_loss = criterion(label_batch_test,embeddings)
        
                
                final_test_loss+=test_loss.detach().item()
                
                if j%100==99:
                    writer.add_scalar('testing loss',test_loss.detach().item(),epoch * len(test_loader) + j)
                
            print("final test_loss: {}".format(final_test_loss/(j+1)))
            
            if final_test_loss/(j+1)<best_val:
                best_val=final_test_loss/(j+1)
                name=BEST_MODEL_PATH+'/model_optim.pth'
                save_model(model,optimizer,name,scheduler=None)
                print("saved model with test dice score: {}".format(best_val))

name=BEST_MODEL_PATH+'/model_final.pth'
save_model(model,optimizer,name,scheduler=None)
        
        

print('Finished Training')



# M2GAN

In [ ]:
class conv_block(nn.Module):
    def __init__(self,channel_in,channel_out,kernel_size,padding,stride):
        super().__init__()
        self.conv=nn.Sequential(nn.Conv2d(channel_in,channel_out,kernel_size=kernel_size,stride=stride,padding=padding,bias=True),
                                nn.BatchNorm2d(channel_out),
                                nn.ReLU(inplace=True))
    
    def forward(self,x):
        x=self.conv(x)
        return x

# class de_conv


In [ ]:

a = torch.randn(4, 4).to(device)
print(a.device)
torch.max(a, 1).values

In [ ]:
torch.max(torch.tensor([1.0]), torch.tensor([0.0]))